<a href="https://colab.research.google.com/github/mightyssge/ING-DATOS/blob/main/Notebooks/ABD_MLlib_Regresion_RegLineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" />  MLlib: Regresión (I)
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>

# Ejemplo 1 de Regresión: Regresión Lineal

In [1]:
!apt-get install -y openjdk-11-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core ca-certificates-java fonts-dejavu-core fonts-dejavu-extra
  gsettings-desktop-schemas java-common libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data libatspi2.0-0
  libpcsclite1 libxcomposite1 libxt-dev libxtst6 libxxf86dga1
  openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
  session-migration x11-utils
Suggested packages:
  default-jre pcscd libxt-doc openjdk-11-demo openjdk-11-source visualvm
  libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  at-spi2-core ca-certificates-java fonts-dejavu-core fonts-dejavu-extra
  gsettings-desktop-schemas java-common libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data libatspi2.0

In [2]:
# Solo si se corre en Google Colab
!pip install -q pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.getOrCreate()

In [4]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/Clientes.csv

## 1.&nbsp;Lectura de Datos

Se examinará el dataset llamado "Clientes" que contiene datos del sitio web y de la aplicación móvil de una compañía. Se desea construir un modelo de regresión que prediga el gasto anual del cliente en los productos de la compañía.

In [5]:
# Cargar los datos
df = spark.read.csv("Clientes.csv", inferSchema=True, header=True)

# Esquema de los datos
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Direccion: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Promedio sesion: double (nullable = true)
 |-- Tiempo en App: double (nullable = true)
 |-- Tiempo en SitioWeb: double (nullable = true)
 |-- Tiempo de Membresia: double (nullable = true)
 |-- Gasto Anual: double (nullable = true)



In [6]:
# Mostrar las 5 primeras filas
df.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+-------------------+------------------+
|               Email|           Direccion|          Avatar|   Promedio sesion|     Tiempo en App|Tiempo en SitioWeb|Tiempo de Membresia|       Gasto Anual|
+--------------------+--------------------+----------------+------------------+------------------+------------------+-------------------+------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|   2.66403418213262| 392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|  4.104543202376424|487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|     SaddleBrow

## 2.&nbsp;Pre-procesamiento

Para regresión con MLlib los datos deben encontrarse en dos columnas: `("etiquetas","atributos")`, donde `atributos` es una sola columna que contiene una lista con todos los atributos. Para poder crear esta columna `atributos` se debe agrupar los atributos usando un `VectorAssembler`.

In [7]:
from pyspark.ml.feature import VectorAssembler

En este ejemplo las variables que se utilizará para predecir será solo las variables cuantitativas (Promedio sesión, Tiempo en App, Tiempo en SitioWeb, Tiempo de Membresía) y se descartará las variables cualitativas que posiblemente no aportan información relevante para la predicción.

In [8]:
# Se utilizará los siguientes atributos bajo el nombre "atributos"
vassembler = VectorAssembler(inputCols=['Promedio sesion',
                                        'Tiempo en App',
                                        'Tiempo en SitioWeb',
                                        'Tiempo de Membresia'],
                             outputCol="Atributos")

# Aplicar a los datos
df2 = vassembler.transform(df)

# Visualizar el resultado
df2.show(5, truncate=False)

+-----------------------------+--------------------------------------------------------+----------------+------------------+------------------+------------------+-------------------+------------------+----------------------------------------------------------------------------+
|Email                        |Direccion                                               |Avatar          |Promedio sesion   |Tiempo en App     |Tiempo en SitioWeb|Tiempo de Membresia|Gasto Anual       |Atributos                                                                   |
+-----------------------------+--------------------------------------------------------+----------------+------------------+------------------+------------------+-------------------+------------------+----------------------------------------------------------------------------+
|mstephenson@fernandez.com    |835 Frank TunnelWrightmouth, MI 82180-9605              |Violet          |34.49726772511229 |12.65565114916675 |39.57766801952616 |4

Se desea predecir el "Gasto Anual" a partir de los atributos anteriores, por lo que el DataFrame que se utilizará contendrá las columnas `Atributos` y `Gasto Anual`.

In [9]:
# Conjunto de datos para regresión: "atributos, gasto anual"
df_final = df2.select("Atributos", 'Gasto Anual')
df_final.show(5, truncate=False)

+----------------------------------------------------------------------------+------------------+
|Atributos                                                                   |Gasto Anual       |
+----------------------------------------------------------------------------+------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]  |587.9510539684005 |
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]  |392.2049334443264 |
|[33.000914755642675,11.330278057777512,37.110597442120856,4.104543202376424]|487.54750486747207|
|[34.30555662975554,13.717513665142507,36.72128267790313,3.120178782748092]  |581.8523440352177 |
|[33.33067252364639,12.795188551078114,37.53665330059473,4.446308318351434]  |599.4060920457634 |
+----------------------------------------------------------------------------+------------------+
only showing top 5 rows



Separación de los datos en un conjunto de entrenamiento `train_data` y de prueba (evaluación) `test_data`. En este caso se utilizará una proporción de 70% a 30%.

In [10]:
train_data, test_data = df_final.randomSplit([0.7, 0.3])

Se mostrará un resumen de los datos de entrenamiento (`train_data`) y de los datos de prueba o evaluación (`test_data`).

In [11]:
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|       Gasto Anual|
+-------+------------------+
|  count|               351|
|   mean| 500.5630885101522|
| stddev|  79.3849574774757|
|    min|256.67058229005585|
|    max| 744.2218671047146|
+-------+------------------+

+-------+-----------------+
|summary|      Gasto Anual|
+-------+-----------------+
|  count|              149|
|   mean|496.3716447129658|
| stddev|79.33843165184298|
|    min| 266.086340948469|
|    max|765.5184619388373|
+-------+-----------------+



## 3.&nbsp;Entrenamiento del Modelo de Regresión

Los datos para cualquier modelo supervisado de Spark MLlib requieren dos columnas: "label" y "features".

* La columna "label" necesita ser numérica, sea de valor real (para una regresión numérica), o de valor entero (para clasificación).
* La columna "feature" contiene un vector con todos los atributos ("features") que corresponden a dicha fila. Usualmente para obtener esta columna se combina varias columnas de atributos en una sola columna

In [12]:
# Creación de un objeto de Regresión Lineal
reglin = LinearRegression(featuresCol="Atributos",
                          labelCol='Gasto Anual',
                          predictionCol= "Predicción")

# Entrenamiento del modelo (con el conjunto de entrenamiento)
modelo = reglin.fit(train_data)

In [13]:
modelo

LinearRegressionModel: uid=LinearRegression_b474b0b57c53, numFeatures=4

In [14]:
# Coeficientes e intercepto para la regresión lineal
print("Coeficientes:", modelo.coefficients)
print("Intercepto:", modelo.intercept)

Coeficientes: [25.314354062494875,38.742812241260154,0.33266565955236244,61.792904424860325]
Intercepto: -1034.9635386264192


El atributo "summary" contiene información sobre el modelo entrenado (evaluación sobre el conjunto de entrenamiento)

In [15]:
# Métricas en el conjunto de entrenamiento
print("MAE del conjunto de entrenamiento:", modelo.summary.meanAbsoluteError)
print("RMSE del conjunto de entrenamiento:", modelo.summary.rootMeanSquaredError)
print("r2 del conjunto de entrenamiento:", modelo.summary.r2)

MAE del conjunto de entrenamiento: 7.898449479246393
RMSE del conjunto de entrenamiento: 10.05060996071822
r2 del conjunto de entrenamiento: 0.9839251296825269


El siguiente DataFrame muestra las predicciones del modelo para el conjunto de entrenamiento.

In [ ]:
df_salida = modelo.summary.predictions
df_salida.show(5)

Se realizará una figura de los valores reales ("Gasto Anual") y los valores predichos ("Predicción") para inspección visual  de la predicción en el conjunto de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Recuperar las columnas de interés: Gasto Anual y Predicción
yreal = df_salida.select('Gasto Anual').collect()
ypred = df_salida.select('Predicción').collect()

# Figuras
plt.figure(figsize=(12, 4))
plt.plot(yreal)
plt.plot(ypred)
plt.legend(['Gasto real', 'Gasto predicho'])
plt.xlabel('Muestra'); plt.ylabel('Gasto');

## 4.&nbsp;Evaluación del Modelo

In [ ]:
# Aplicación del modelo entrenado al conjunto de prueba (test)
resultados = modelo.evaluate(test_data)
resultados

Resultados en el conjunto de evaluación:
* Gasto Anual: etiquetas reales
* Predicción: predicciones realizadas por el modelo

In [ ]:
resultados.predictions.show(5, truncate=False)

In [ ]:
# Residuos (valores "reales" - valores "predichos")
resultados.residuals.show(8)

In [ ]:
# Métricas en el conjunto de evaluación
print("MAE del conjunto de prueba:", resultados.meanAbsoluteError)
print("RMSE del conjunto de prueba:", resultados.rootMeanSquaredError)
print("MSE del conjunto de prueba:", resultados.meanSquaredError)
print("R^2 del conjunto de prueba:", resultados.r2)

Inspección visual de la predicción en el conjunto de prueba o evaluación.



In [ ]:
# Recuperar las columnas de interés: Gasto Anual y Predicción
yreal = resultados.predictions.select('Gasto Anual').collect()
ypred = resultados.predictions.select('Predicción').collect()

# Figuras
plt.figure(figsize=(12, 4))
plt.plot(yreal)
plt.plot(ypred)
plt.legend(['Gasto real', 'Gasto predicho'])
plt.xlabel('Muestra'); plt.ylabel('Gasto');

## 5.&nbsp;Predicción

Una vez entrenado el modelo, se puede utilizar para predecir valores para nuevos conjuntos de datos. En este caso, para realizar esta prueba, se utilizará únicamente la columna de `Atributos` del conjunto de evaluación.

In [ ]:
# Datos para los cuales se realizará la predicción
df2 = test_data.select('Atributos')
df2.show(5, truncate=False)

In [ ]:
# Predicciones sobre los datos
predicciones = modelo.transform(df2)

# Resultado
predicciones.show(5, truncate=False)